## Grabbing historical weather data for 2018

In [2]:
##load variables

import os
from dotenv import load_dotenv
from darksky import forecast
from datetime import date,datetime, timedelta
import pandas as pd


##read in secret keys stored in .env file
load_dotenv('.env')
SECRET = os.environ['dark_sky_secret']
LAT = os.environ['LAT']
LONG = os.environ['LONG']


In [8]:
##test api call
t = datetime(2013, 5, 6, 12).isoformat()
data = forecast(SECRET,LAT,LONG, time=t)
data.time

1367856000

In [68]:
##api call

# sdate = datetime(2018, 1, 1,12)  # start date
# edate = datetime(2018, 12, 31,12)   # end date

# delta = edate - sdate       # as timedelta
# weather = {}
# for i in range(delta.days + 1):
#     day = sdate + timedelta(days=i)
#     key = day.strftime('%D')
#     day = day.isoformat()
#     test = forecast(SECRET,LAT,LONG, time=day)
#     weather[key] = test


In [108]:
all_dates = []
test_df = pd.DataFrame()
for k, v in weather.items():
    toy_df = pd.DataFrame()
    for i in range(0,22):
        all_dates.append(k)
        timei = pd.DataFrame.from_dict(weather[k]['hourly']['data'][i],orient='index').transpose()
        timei['datetime'] =  pd.to_datetime(weather[k]['hourly']['data'][i]['time'],unit='s')
        toy_df= pd.concat([toy_df,timei], sort = False)
    test_df = pd.concat([test_df, toy_df], sort = False)
        
        

### Building test case for App 

In [111]:
# import pickle
# with open('data/weather2018.pkl', 'wb') as fid:
#     pickle.dump(test_df, fid,2) 

In [161]:
import pickle
weather = pickle.load(open('data/weather2018.pkl', 'rb'))
weather = pd.DataFrame(weather)

head= pd.read_csv('data/head.csv')
appt_date = "AppointmentDate"

head.AppointmentDate = (pd.to_datetime(head.AppointmentDate) + 
                                (pd.Timedelta(days = 1) - pd.Timedelta(seconds = 1)) * (head.noshow))


In [157]:
##test api call
t = pd.to_datetime(head.AppointmentDate[1]).isoformat()
data = forecast(SECRET,LAT,LONG, time=t)
pd.to_datetime(data.time, unit='s')

Timestamp('2018-06-21 13:00:00')

In [152]:
#fucntion that gets weather data given appointment data and name of date column
def get_weather(data, appointment_date):
    forecasts = {}
    for i in data[appointment_date]:
        day = pd.to_datetime(i)
        key = day.strftime('%D')
        day = day.isoformat()
        test = forecast(SECRET,LAT,LONG, time=day)
        forecasts[key] = test
    return forecasts


KeyError: 'AppointmentDate'

In [153]:
test_weather_dict = get_weather(head, appt_date)

HTTPError: Bad response

In [125]:
def convert_weather_to_pd(weather_df):
    all_dates = []
    container_df = pd.DataFrame()
    for k, v in weather_df.items():
        per_appointment = pd.DataFrame()
        for i in range(0,22):
            all_dates.append(k)
            per_hour = pd.DataFrame.from_dict(weather_df[k]['hourly']['data'][i],orient='index').transpose()
            per_hour['datetime'] =  pd.to_datetime(weather_df[k]['hourly']['data'][i]['time'],unit='s')
            per_appointment= pd.concat([per_appointment,per_hour], sort = False)
    container_df = pd.concat([container_df, per_appointment], sort = False)
    return container_df

In [148]:
test_weather_pd = convert_weather_to_pd(test_weather_dict)

In [143]:
def merge_appt_weather(appointments_df,weather_df, appt_date):#takes appointments, weather, and column of appointment date
    x = appointments_df[appt_date].dt.round('60min')
    appointments_df['datetime'] = pd.to_datetime(x)
    weather_df['datetime'] = pd.to_datetime(weather_df['time'],unit='s').dt.round('60min')
    appointments_df = appointments_df.merge(weather_df,on='datetime', how="left")
    return appointments_df

In [160]:
x = head[appt_date].dt.round('60min')
head['datetime'] = pd.to_datetime(x)
test_weather_pd['datetime'] = pd.to_datetime(test_weather_pd['time'],unit='s').dt.round('60min')
head = head.merge(test_weather_pd,on='datetime', how="left")
head

,Unnamed: 0,PatientId,AppointmentDate,newbie,insuranceDummy,guarantorIsPatient,loyalty,noshow,PatientEmail,GuarantorEmail,...,dewPoint_y,humidity_y,pressure_y,windSpeed_y,windGust_y,windBearing_y,cloudCover_y,uvIndex_y,visibility_y,ozone_y
0,0,5308,2018-09-26 14:30:00,0,0,True,27,0,daviddm75@gmail.com,daviddm75@gmail.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,9552,2018-06-21 09:00:00,0,1,True,13,0,north.virginia@gmail.com,north.virginia@gmail.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,6882,2018-06-12 08:00:00,0,0,True,24,0,missericab88@aol.com,missericab88@aol.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,9637,2018-02-06 08:00:00,0,0,True,20,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,10143,2018-06-20 12:00:00,0,0,False,24,0,samadi@vt.edu,SS3AQ@VIRGINIA.EDU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,10283,2018-08-28 08:00:00,0,1,True,4,0,dinacruz0809@gmail.com,dinacruz0809@gmail.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,9789,2018-07-31 15:00:00,0,0,False,25,0,NaN,opt out,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,4093,2018-06-13 12:00:00,0,0,True,24,0,smremley@comcast.net,smremley@comcast.net,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,9119,2018-10-18 23:59:58,0,1,True,34,1,etroche8@gmail.com,etroche8@gmail.com,...,38.35,0.69,1030.03,0.72,0.72,297,0,0,7.541,296.1


In [162]:
merge_appt_weather(head, weather,appt_date )

,Unnamed: 0,PatientId,AppointmentDate,newbie,insuranceDummy,guarantorIsPatient,loyalty,noshow,PatientEmail,GuarantorEmail,...,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,precipType,precipAccumulation,ozone
0,0,5308,2018-09-26 14:30:00,0,0,True,27,0,daviddm75@gmail.com,daviddm75@gmail.com,...,1016.71,5.94,5.94,182,0.55,3,7.197,NaN,NaN,273.1
1,1,9552,2018-06-21 09:00:00,0,1,True,13,0,north.virginia@gmail.com,north.virginia@gmail.com,...,1007.07,1.15,1.15,207,0.43,0,5.282,NaN,NaN,NaN
2,2,6882,2018-06-12 08:00:00,0,0,True,24,0,missericab88@aol.com,missericab88@aol.com,...,1022.58,1.06,1.08,82,0,0,4.326,NaN,NaN,NaN
3,3,9637,2018-02-06 08:00:00,0,0,True,20,0,NaN,NaN,...,1026.58,0.02,0.16,NaN,0.03,0,7.462,NaN,NaN,NaN
4,4,10143,2018-06-20 12:00:00,0,0,False,24,0,samadi@vt.edu,SS3AQ@VIRGINIA.EDU,...,1012.4,2.3,2.3,16,0.68,1,7.431,NaN,NaN,NaN
5,5,10283,2018-08-28 08:00:00,0,1,True,4,0,dinacruz0809@gmail.com,dinacruz0809@gmail.com,...,1018.66,0.44,0.44,204,0.05,0,2.455,NaN,NaN,NaN
6,6,9789,2018-07-31 15:00:00,0,0,False,25,0,NaN,opt out,...,1019.6,4.9,4.9,102,0.98,4,7.368,NaN,NaN,NaN
7,7,4093,2018-06-13 12:00:00,0,0,True,24,0,smremley@comcast.net,smremley@comcast.net,...,1016.07,5.13,5.13,192,0.84,1,6.599,NaN,NaN,NaN
8,8,9119,2018-10-18 23:59:58,0,1,True,34,1,etroche8@gmail.com,etroche8@gmail.com,...,1029.97,0.72,0.72,297,0,0,7.541,NaN,NaN,296.1
